In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from datetime import datetime
import numpy as np
import json

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [2]:
%%time
df_2021 = pd.read_csv('../전처리/wi-fi/path/presence_path_2021.csv', encoding='cp949')

CPU times: total: 1min 24s
Wall time: 1min 25s


In [2]:
%%time
df_2022 = pd.read_csv('../전처리/wi-fi/path/presence_path_2022.csv', encoding='cp949')

CPU times: total: 33.7 s
Wall time: 35.4 s


In [3]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
# 데이터프레임 타입변경
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [5]:
print('df_2021')
df_astype(df_2021)
print('-------------'*3)

df_2021
데이터 프레임의 메모리 사용량 : 2705.92 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 1646.35 MB
감소율 : 39.2%
---------------------------------------


In [5]:
print('df_2022')
df_astype(df_2022)
print('-------------'*3)

df_2022
데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%
---------------------------------------


In [6]:
# 동일한 위경도값을 보유한 포인트를 1개의 개소로 측정

# 위경도 csv 파일 읽기
wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])


# MAC 코드화 딕셔너리 불러오기
with open('../mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))

# 사용되지 않은 Wi-Fi, 목록에서 제거
for n in wk.index:
    if n not in mac_code.values():
        wk.drop(n, axis=0, inplace=True)

# 위도로 정렬 (같은 위도 묶기)
wk.sort_values('위도', inplace=True)

# 이전 Mac 주소를 담을 변수
f_mac = None

# 동일한 위경도를 보유한 Mac 주소 리스트
repeat = []

# repeat 리스트를 모은 리스트
repeat_all = []

for n in wk.index:
    # 첫행 Skip
    if n == wk.iloc[0].name:
        f_mac = n
        continue
    # 이전 행 (위도, 경도)와 같다면 append
    if (wk.loc[f_mac]['위도'] == wk.loc[n]['위도']) and (wk.loc[f_mac]['경도'] == wk.loc[n]['경도']):
        repeat.append(f_mac)
        repeat.append(n)
    
    # 전체 리스트 
    else:
        # 빈 리스트는 무시
        if len(repeat) == 0:
            f_mac = n
            continue
        repeat_all.append(list(set(repeat)))
        repeat = []
    f_mac = n
    
with open('../mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

repeat_num_all = []
repeat_num = []
for n in repeat_all:
    for i in range(len(n)):
        repeat_num.append(mac_code[n[i]])
    repeat_num_all.append(repeat_num)
    repeat_num = []
    
    
# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))


for n in repeat_num_all:
    for i in range(len(n)):
        if i == 0:
            continue
        mac_code[n[i]] = mac_code[n[0]]

In [136]:
def mac_ch(df):
    df.reset_index(drop=True, inplace=True)
    for x in tqdm(range(len(df))):
        df['MAC_add'][x] = mac_code[df['MAC_add'][x]]

In [30]:
mac_ch(df_2021)

  0%|          | 0/44333789 [00:00<?, ?it/s]

In [137]:
mac_ch(df_2022)

  0%|          | 0/28919721 [00:00<?, ?it/s]

In [31]:
df_2021.to_csv('../전처리/wi-fi/MAC변환/presence_path_2021.csv', encoding='cp949', index=False)
df_2022.to_csv('../전처리/wi-fi/MAC변환/presence_path_2022.csv', encoding='cp949', index=False)

In [8]:
df_2021 = pd.read_csv('../전처리/wi-fi/MAC변환/presence_path_2021.csv', encoding='cp949')
df_2022 = pd.read_csv('../전처리/wi-fi/MAC변환/presence_path_2022.csv', encoding='cp949')

In [ ]:
print('df_2021')
df_astype(df_2021)
print('-------------'*3)

In [9]:
print('df_2022')
df_astype(df_2022)
print('-------------'*3)

df_2022
데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%
---------------------------------------


In [88]:
# 21년 8월달 추출
i = 8
df = df_2021[(df_2021['STARTTIMESTAMP'] >= datetime.strptime(f'2021-{i}-01 00:00:00',
                                              '%Y-%m-%d %H:%M:%S')) &
    (df_2021['STARTTIMESTAMP'] < datetime.strptime(f'2021-{i+1}-01 00:00:00',
                                              '%Y-%m-%d %H:%M:%S'))]


df_mac_list = df['MAC_add'].unique()

cnct_df = None

for n in tqdm(df_mac_list):
    DF = df[df['MAC_add']==n]
    DF = pd.DataFrame(DF.groupby('MAC_add')['CLIENTMAC'].value_counts())
    #DF = DF[(DF['CLIENTMAC']>0)&(DF['CLIENTMAC']<3)]
    DF = DF[DF['CLIENTMAC'] == 1]
    DF.rename(columns={'CLIENTMAC':'use_count'}, inplace=True)
    
    
    
    if n == df_mac_list[0]:
        cnct_df = DF
        continue
    cnct_df = pd.concat([cnct_df, DF])

  0%|          | 0/53 [00:00<?, ?it/s]

In [89]:
cnct_df.rename(columns={'CLIENTMAC':'use_count'}, inplace=True)

In [90]:
cnct_df.reset_index(inplace=True)

In [66]:
# 50회 이상 이용자
cnct_df

MAC_add     CLIENTMAC  use_count
0     7C573CCFB76A  84701FDCA5D5       1975
1     7C573CCFB76A  DAA119A8E654       1876
2     7C573CCFB76A  286D97AAD19B        672
3     7C573CCFB76A  000918E16027        570
4     7C573CCFB76A  0009745795F5        551
...            ...           ...        ...
3525  7C573CCF4352  88571D6CC2E4        362
3526  7C573CCF4352  88571D584B60        239
3527  7C573CCF4352  7440BE46B5F9        171
3528  7C573CCF4352  C097276EF6B9         60
3529  7C573CCF4352  203DBD9F6FB9         56

[3530 rows x 3 columns]

In [58]:
# 1회 이용자
cnct_df

MAC_add     CLIENTMAC  use_count
0       7C573CCFB76A  000CE711A04E          1
1       7C573CCFB76A  EC533E7A44A2          1
2       7C573CCFB76A  ECAA258A8A45          1
3       7C573CCFB76A  50770512D927          1
4       7C573CCFB76A  948BC13CB4E6          1
...              ...           ...        ...
365227  7C573CCF4352  F08A76DF57E4          1
365228  7C573CCF4352  8C69F6AFBA90          1
365229  7C573CCF4352  8086D96EEEB2          1
365230  7C573CCF4352  000CE7B0CC44          1
365231  7C573CCF4352  6C0B84BEF0D1          1

[365232 rows x 3 columns]

In [39]:
# 1~2회 이용자
cnct_df

MAC_add     CLIENTMAC  use_count
0       7C573CCFB76A  F8E61AFD564B          2
1       7C573CCFB76A  08AED6FACDD9          2
2       7C573CCFB76A  0C2FB0EACEFC          2
3       7C573CCFB76A  A82BB9E57E9A          2
4       7C573CCFB76A  7C89568BEB94          2
...              ...           ...        ...
415729  7C573CCF4352  F08A76DF57E4          1
415730  7C573CCF4352  8C69F6AFBA90          1
415731  7C573CCF4352  8086D96EEEB2          1
415732  7C573CCF4352  000CE7B0CC44          1
415733  7C573CCF4352  6C0B84BEF0D1          1

[415734 rows x 3 columns]

In [91]:
cnct_df = pd.DataFrame(cnct_df.groupby('MAC_add')['CLIENTMAC'].count())
cnct_df.reset_index(inplace=True)

In [92]:
cnct_df.rename(columns={'CLIENTMAC':'user_count'}, inplace=True)
cnct_df

MAC_add  user_count
0   7C573CCF42BE        8007
1   7C573CCF4320        6783
2   7C573CCF4342        1846
3   7C573CCF4352        1855
4   7C573CCF4398       10507
5   7C573CCF43DE        1456
6   7C573CCF43EE       11802
7   7C573CCF44C0        1137
8   7C573CCF44DA        5604
9   7C573CCF4518       17687
10  7C573CCF4526        2593
11  7C573CCF454E       12307
12  7C573CCF455A        2258
13  7C573CCF456E       11490
14  7C573CCF457A        6436
15  7C573CCF45BA        8090
16  7C573CCF45F4        4041
17  7C573CCF465E       13355
18  7C573CCF46C8       17978
19  7C573CCF470A        8752
20  7C573CCF471C        3232
21  7C573CCF4740        4678
22  7C573CCF47D0        7159
23  7C573CCF47E6        2933
24  7C573CCF484E        6557
25  7C573CCF48D4       11875
26  7C573CCF497E       12031
27  7C573CCF49AA        4148
28  7C573CCF49C0        2198
29  7C573CCF49F8        1772
30  7C573CCF4A94        2184
31  7C573CCF4AAE         977
32  7C573CCF4AD6        1314
33  7C573CCF4AE6       15648
34  7C573CCF4AEE        7528
35  7C573CCF4B2E        2220
36  7C573CCF4B36       28027
37  7C573CCF4B7C        1217
38  7C573CCF4B80        5766
39  7C573CCF4B98        3456
40  7C573CCF4CCC        6955
41  7C573CCF4CE8        9587
42  7C573CCF4D0C        6260
43  7C573CCF4D6A        3840
44  7C573CCF4D74        3449
45  7C573CCF4E8C        5743
46  7C573CCF4EA6        3907
47  7C573CCF4ECC        6106
48  7C573CCF4EF2        7160
49  7C573CCF4F78       12633
50  7C573CCF4F88        4038
51  7C573CCFB76A       16119
52  D015A6C7F506         531

In [93]:
# 폴더 생성
try:
    directory = f'../전처리/wi-fi/월별_사용자/1회_사용자/2021년/'
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print ('Error: Creating directory. ' +  directory)

cnct_df.to_csv(f'{directory}{i}월.csv', encoding='cp949', index=False)

In [39]:
wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])
wk

위도          경도
MAC주소                              
7C573CCF470A  35.249417  128.869694
7C573CCF4526  35.247250  128.868417
7C573CCF4A7E  35.247056  128.868472
7C573CCF4D6A  35.247056  128.868472
7C573CCF45F4  35.246833  128.867889
...                 ...         ...
7C573CCF4BD4  35.228722  128.871500
7C573CCFB76A  35.227438  128.873926
7C573CCF48D4  35.227167  128.873806
7C573CCF4E54  35.227111  128.875167
7C573CCF4AE6  35.227111  128.875167

[74 rows x 2 columns]

In [69]:
cnct_df['X'] = None
cnct_df['Y'] = None

for n in range(len(cnct_df)):
    cnct_df['X'][n] = wk.loc[cnct_df['MAC_add'][n]]['경도']
    cnct_df['Y'][n] = wk.loc[cnct_df['MAC_add'][n]]['위도']

In [70]:
cnct_df.to_csv('./전처리/50회_이상_와이파이_이용자/2021년/8월.csv', encoding='cp949', index=False)

In [14]:
def N_user_counting(year, df_year):

    # 위경도 csv 파일 읽기
    wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])
    
    all_df = None
    cnct_df = None

    
    # 월 리스트 생성
    if year == 2021:
        month_list = [m for m in range(1,13)]
    
    elif year == 2022:
        month_list = [m for m in range(3,11)]
    
    
    print(f'{year}년 전체 진행도')
    for i in tqdm(month_list):
        print(f'{i}월 진행중')
        if i != month_list[-1]:
            df = df_year[(df_year['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{i}-01 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S')) &
                (df_year['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{i+1}-01 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S'))]
        else:
            df = df_year[(df_year['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{i}-01 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S'))]


        df_mac_list = df['MAC_add'].unique()

    
        for n in tqdm(df_mac_list):
            DF = df[df['MAC_add']==n]
            DF = pd.DataFrame(DF.groupby('MAC_add')['CLIENTMAC'].value_counts())
            #DF = DF[(DF['CLIENTMAC']>0)&(DF['CLIENTMAC']<3)]
            DF = DF[DF['CLIENTMAC'] == 1]

            if n == df_mac_list[0]:
                cnct_df = DF
                continue
            cnct_df = pd.concat([cnct_df, DF])


        cnct_df.rename(columns={'CLIENTMAC':'use_count'}, inplace=True)
        cnct_df.reset_index(inplace=True)
        cnct_df = pd.DataFrame(cnct_df.groupby('MAC_add')['CLIENTMAC'].count())
        cnct_df.reset_index(inplace=True)
        cnct_df.rename(columns={'CLIENTMAC':'user_count'}, inplace=True)


        
        # 위경도 추가
        cnct_df['X'] = None
        cnct_df['Y'] = None

        for n in range(len(cnct_df)):
            cnct_df['X'][n] = wk.loc[cnct_df['MAC_add'][n]]['경도']
            cnct_df['Y'][n] = wk.loc[cnct_df['MAC_add'][n]]['위도']
            
            

        # 폴더 생성
        try:
            directory = f'../전처리/wi-fi/월별_사용자/1회_사용자/{year}년/'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)

        cnct_df.to_csv(f'{directory}{i}월.csv', encoding='cp949', index=False)


        if i == month_list[0]:
            all_df = cnct_df
            continue
        all_df = pd.concat([all_df, cnct_df])
        
    all_df.to_csv(f'{directory}{year}년.csv', encoding='cp949', index=False)

In [15]:
N_user_counting(2022, df_2022)

2022년 전체 진행도


  0%|          | 0/8 [00:00<?, ?it/s]

3월 진행중


  0%|          | 0/53 [00:00<?, ?it/s]

4월 진행중


...

  0%|          | 0/53 [00:00<?, ?it/s]

9월 진행중


  0%|          | 0/53 [00:00<?, ?it/s]

10월 진행중


  0%|          | 0/53 [00:00<?, ?it/s]

# 연도별 (Wi-Fi 개소별 월 1회 이용자)

In [59]:
def read_month_1user(year:int):
    
    # 연도별 월 리스트 생성
    if year == 2021:
        month_list = [n for n in range(1,13)]
    elif year == 2022:
        month_list = [n for n in range(3,11)]
    
    # 월별 파일 불러오기
    for i in tqdm(month_list):    
        cnct_df = pd.read_csv(f'../전처리/wi-fi/월별_사용자/1회_사용자/{year}년/{i}월.csv',
                                      encoding='cp949')
        cnct_df['year'] = year
        cnct_df['month'] = i

        if i == 1:
            globals()[f'all_df_{year}'] =  cnct_df
            continue
        else:
            globals()[f'all_df_{year}'] = pd.concat([globals()[f'all_df_{year}'], cnct_df])
    print(f'all_df_{year} 변수 생성 완료')

In [62]:
read_month_1user(2021)
read_month_1user(2022)

  0%|          | 0/12 [00:00<?, ?it/s]

all_df_2021 변수 생성 완료


  0%|          | 0/8 [00:00<?, ?it/s]

all_df_2022 변수 생성 완료


In [ ]:
read_month_1user()

### 엑셀파일 저장

In [25]:
pdf_sum = pd.concat([all_df_2021, all_df_2022])

pdf1 = pd.pivot_table(pdf_sum,                # 피벗할 데이터프레임
                     index = 'MAC_add',    # 행 위치에 들어갈 열
                     columns = ['year','month'],    # 열 위치에 들어갈 열
                     values = 'user_count')

# NaN값 0으로 처리
pdf1.fillna(0, inplace=True)

In [26]:
# 타입 변환을 위한 딕셔너리 생성
col_astype = dict()
for n in pdf1.columns:
    col_astype[n] = int

# 타입 변환
pdf1 = pdf1.astype(col_astype)

# MAC 주소를 컬럼으로 빼냄
pdf1.reset_index(inplace=True)

In [31]:
# 설치 장소, 위치 추가를 위한 데이터 불러오기
wifi = pd.read_csv('./원본/테마형특화단지ap_upper.csv', encoding='cp949')

# 설치 장소, 위치 추가
pdf1['설치장소명'] = None
pdf1['설치위치'] = None

for n in range(len(pdf1)):
    pdf1[('설치장소명','')][n] = list(wifi[wifi['MAC주소']==pdf1['MAC_add'][n]]['설치 장소명'])[0]
    pdf1[('설치위치','')][n] = list(wifi[wifi['MAC주소']==pdf1['MAC_add'][n]]['설치 위치'])[0]

    
# 컬럼명 순서 재배치를 위한 리스트 생성    
col_list_1 = [('설치장소명', ''), ('설치위치', '')]
col_list_2 = list(pdf1.columns[:-2])
col_list_3 = col_list_1 + col_list_2

# 컬럼명 순서 재배치
pdf1 = pdf1[col_list_3]

# 엑셀 저장
pdf1.to_excel('./전처리/와이파이_1회_이용자.xlsx')

### 연도별 (Wi-Fi 개소별 월 1회 이용자) 좌표값 

In [49]:
def year_point_1user(year:int, DF):
    # 동일한 개소 병합
    all_df_group = pd.DataFrame(DF.groupby('MAC_add')['user_count'].sum())
    
    # 인덱스 초기화
    all_df_group.reset_index(inplace=True)
    
    # X,Y 좌표 컬럼 생성
    all_df_group['X'] = None
    all_df_group['Y'] = None
    
    # 좌표 추가
    for n in tqdm(range(len(all_df_group))):
        all_df_group['X'][n] = wk.loc[all_df_group['MAC_add'][n]]['경도']
        all_df_group['Y'][n] = wk.loc[all_df_group['MAC_add'][n]]['위도']
    
    all_df_group.to_csv(f'../전처리/wi-fi/월별_사용자/1회_사용자/{year}년/{year}년.csv', encoding='cp949', index=False)
    print(f'{year}년 CSV파일 저장 완료')

In [50]:
year_point_1user(2022, all_df_2022)

  0%|          | 0/53 [00:00<?, ?it/s]

2022년 CSV파일 저장 완료


# -------------------------------------------------------------------------------------------------------------

In [58]:
def month_split(DF, year, month_list):
    
    global concat_df
    for n in tqdm(month_list):
        
        ye = []
        mon = []
        da = []
        con =[]
        
        if n <= 9:
            num = '0' + str(n)
        else:
            num = str(n)

        # 기간설정(월별)
        if n != 12:
            df = DF[(DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (DF['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{n+1}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S'))]
        else:
            df = DF[DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00', 
                                                              '%Y-%m-%d %H:%M:%S')]
            
        # 마지막 일    
        max_day = df['STARTTIMESTAMP'].dt.day.max()
        
        
        df.reset_index(drop=True, inplace=True)
        for x in tqdm(range(len(df))):
            df['MAC_add'][x] = mac_code[df['MAC_add'][x]]
            if np.isnan(df['Next_MAC'][x]):
                continue
            else:
                df['Next_MAC'][x] = mac_code[df['Next_MAC'][x]]
        
        
        for day in range(1, max_day+1):
            
            # 기간설정(일별)
            if day != max_day:
                df_d = df[(df['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-{day} 00:00:00',
                                                                  '%Y-%m-%d %H:%M:%S')) &
                        (df['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{n}-{day+1} 00:00:00',
                                                                  '%Y-%m-%d %H:%M:%S'))]
            else:
                df_d = df[df['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-{day} 00:00:00', 
                                                                  '%Y-%m-%d %H:%M:%S')]
            ye.append(year)
            mon.append(n)
            da.append(day)
            con.append(len(df_d['CLIENTMAC'].unique()))
            
            
        sum_list = [ye, mon,da,con]    
        
        df_new = pd.DataFrame(sum_list, columns=['yaer', 'month', 'day', 'user_count'])
        
        if n == month_list[0]:
            concat_df = df_new
            continue
        concat_df = pd.concat([concat_df, df_new])
    return concat_df

In [59]:
concat_df

yaer  month           MAC  user_count
0    2021      1  7C573CCF4C04      115492
1    2021      1  7C573CCF4CE8       30796
2    2021      1  7C573CCF4E54       54855
3    2021      1  7C573CCF45BA       73058
4    2021      1  7C573CCF465E      181237
..    ...    ...           ...         ...
847  2021     12  7C573CCF4B80          15
848  2021     12  7C573CCF4B80          14
849  2021     12  7C573CCF4A7E          14
850  2021     12  7C573CCF4B80          17
851  2021     12  D015A6C7F4F2           3

[852 rows x 4 columns]

# 월별 총 User

In [63]:
def month_split(DF, year):
    
    if year == 2021:
        month_list = [n for n in range(1,13)]
    elif year == 2022:
        month_list = [n for n in range(3,11)]
    
    
    global concat_df
    for n in tqdm(month_list):
        
        ye = []
        mon = []
        con =[]
        mc=[]
        
        if n <= 9:
            num = '0' + str(n)
        else:
            num = str(n)

        # 기간설정(월별)
        if n != month_list[-1]:
            df = DF[(DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (DF['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{n+1}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S'))]
        else:
            df = DF[DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00', 
                                                              '%Y-%m-%d %H:%M:%S')]
            
    
        for mac in list(df['MAC_add'].unique()):
            
            
            df_d = df[df['MAC_add']==mac]
            
            ye.append(year)
            mon.append(n)
            mc.append(mac)
            con.append(len(df_d['CLIENTMAC'].unique()))
        
        
        sum_list = []
        for a,b,c,d in zip(ye, mon, mc, con):
            sum_list.append([a,b,c,d])    
        
        df_new = pd.DataFrame(sum_list, columns=['yaer', 'month', 'MAC', 'user_count'])
        
        if n == month_list[0]:
            concat_df = df_new
            continue
        concat_df = pd.concat([concat_df, df_new])
    return concat_df

In [111]:
month_split(df_2021, 2021)

  0%|          | 0/12 [00:00<?, ?it/s]

yaer  month           MAC  user_count
0   2021      1  7C573CCF4B36      261671
1   2021      1  7C573CCF4CE8       72957
2   2021      1  7C573CCF4AE6      107771
3   2021      1  7C573CCF45BA       73058
4   2021      1  7C573CCF465E      181237
..   ...    ...           ...         ...
48  2021     12  7C573CCF4B80          61
49  2021     12  7C573CCF4342          31
50  2021     12  7C573CCF4526          11
51  2021     12  7C573CCF4D6A          30
52  2021     12  D015A6C7F506           3

[629 rows x 4 columns]

In [145]:
month_split(df_2022, 2022)

  0%|          | 0/8 [00:00<?, ?it/s]

yaer  month           MAC  user_count
0   2022      3  7C573CCF465E       18486
1   2022      3  7C573CCF454E       18633
2   2022      3  7C573CCF4518       25339
3   2022      3  7C573CCF49C0        5355
4   2022      3  7C573CCF4B2E        4305
..   ...    ...           ...         ...
48  2022     10  7C573CCF44DA        8713
49  2022     10  7C573CCF4AAE        2631
50  2022     10  7C573CCF4352        4179
51  2022     10  7C573CCF471C        6403
52  2022     10  7C573CCF4B98        7277

[423 rows x 4 columns]

In [148]:
concat_df.reset_index(drop=True,inplace=True)
concat_df

yaer  month           MAC  user_count
0    2022      3  7C573CCF465E       18486
1    2022      3  7C573CCF454E       18633
2    2022      3  7C573CCF4518       25339
3    2022      3  7C573CCF49C0        5355
4    2022      3  7C573CCF4B2E        4305
..    ...    ...           ...         ...
418  2022     10  7C573CCF44DA        8713
419  2022     10  7C573CCF4AAE        2631
420  2022     10  7C573CCF4352        4179
421  2022     10  7C573CCF471C        6403
422  2022     10  7C573CCF4B98        7277

[423 rows x 4 columns]

In [120]:
for mon in range(1,13):
    concat_df_month = concat_df[concat_df['month']==mon]
    concat_df_month.reset_index(drop=True, inplace=True)

    concat_df_month['X'] = None
    concat_df_month['Y'] = None

    for n in tqdm(range(len(concat_df_month))):
        concat_df_month['X'][n] = wk.loc[concat_df_month['MAC'][n]]['경도']
        concat_df_month['Y'][n] = wk.loc[concat_df_month['MAC'][n]]['위도']

    concat_df_month.to_csv(f'./전처리/와이파이_이용자/2021년/{mon}월.csv', encoding='cp949', index=False)

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

...

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

In [149]:
for mon in range(3,11):
    concat_df_month = concat_df[concat_df['month']==mon]
    concat_df_month.reset_index(drop=True, inplace=True)

    concat_df_month['X'] = None
    concat_df_month['Y'] = None

    for n in tqdm(range(len(concat_df_month))):
        concat_df_month['X'][n] = wk.loc[concat_df_month['MAC'][n]]['경도']
        concat_df_month['Y'][n] = wk.loc[concat_df_month['MAC'][n]]['위도']

    concat_df_month.to_csv(f'./전처리/와이파이_이용자/2022년/{mon}월.csv', encoding='cp949', index=False)

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

In [102]:
all_df_2021 = None

for i in range(1,13):    
    cnct_df = pd.read_csv(f'./전처리/와이파이_이용자/2021년/{i}월.csv',
                                  encoding='cp949')
    
    if i == 1:
        all_df_2021 = cnct_df
        continue
    else:
        all_df_2021 = pd.concat([all_df_2021, cnct_df])

In [103]:
all_df_2022 = None

for i in range(3,11):    
    cnct_df = pd.read_csv(f'./전처리/와이파이_이용자/2022년/{i}월.csv',
                                  encoding='cp949')
    
    if i == 3:
        all_df_2022 = cnct_df
        continue
    else:
        all_df_2022 = pd.concat([all_df_2022, cnct_df])

In [108]:
pdf_sum = pd.concat([all_df_2021, all_df_2022])

pdf1 = pd.pivot_table(pdf_sum,                # 피벗할 데이터프레임
                     index = 'MAC',    # 행 위치에 들어갈 열
                     columns = ['yaer','month'],    # 열 위치에 들어갈 열
                     values = 'user_count')


pdf1.fillna(0, inplace=True)

In [112]:
pdf1.reset_index(inplace=True)

In [113]:
pdf1['설치장소명'] = None
pdf1['설치위치'] = None

for n in range(len(pdf1)):
    pdf1[('설치장소명','')][n] = list(wifi[wifi['MAC주소']==pdf1['MAC'][n]]['설치 장소명'])[0]
    pdf1[('설치위치','')][n] = list(wifi[wifi['MAC주소']==pdf1['MAC'][n]]['설치 위치'])[0]

In [114]:
pdf1

yaer            MAC      2021                                                \
month                       1         2        3        4        5        6   
0      7C573CCF42BE  125978.0  111950.0  35313.0  30353.0  28190.0  20222.0   
1      7C573CCF4320  110287.0   89073.0  35157.0  30972.0  27394.0  19446.0   
2      7C573CCF4342   51356.0   49993.0  11296.0   8753.0   7669.0   4649.0   
3      7C573CCF4352   17519.0   15050.0   6170.0   6420.0   5572.0   4278.0   
4      7C573CCF4398  166263.0  136664.0  43033.0  37423.0  35763.0  25187.0   
5      7C573CCF43DE   27394.0   27183.0  13608.0   8550.0   6889.0   4540.0   
6      7C573CCF43EE  106332.0   85276.0  34279.0  29041.0  29435.0  21061.0   
7      7C573CCF44C0   25557.0   23355.0  12011.0   7252.0   5690.0   3835.0   
8      7C573CCF44DA   98700.0   92083.0  25790.0  24935.0  22934.0  15443.0   
9      7C573CCF4518  256904.0  200496.0  71169.0  62734.0  57285.0  42392.0   
10     7C573CCF4526   29918.0   25746.0  12606.0   9373.0   9100.0   5517.0   
11     7C573CCF454E  159470.0  133509.0  53411.0  48937.0  43943.0  30393.0   
12     7C573CCF455A   37645.0   38276.0  16499.0  11956.0   8637.0   5812.0   
13     7C573CCF456E  169291.0  133342.0  43992.0  37769.0  37262.0  25604.0   
14     7C573CCF457A  133154.0  110691.0  32393.0  26810.0  23601.0  17143.0   
15     7C573CCF45BA   73058.0   73173.0  26623.0  30250.0  23556.0  19797.0   
16     7C573CCF45F4   40397.0   36261.0  19280.0  13777.0  12142.0   8156.0   
17     7C573CCF465E  181237.0  138608.0  60343.0  52173.0  47857.0  32477.0   
18     7C573CCF46C8  104599.0   90704.0  40243.0  43982.0  44683.0  33797.0   
19     7C573CCF470A  113725.0   97683.0  41397.0  33300.0  30621.0  20770.0   
20     7C573CCF471C   25919.0   22617.0  10676.0  11527.0  10008.0   5982.0   
21     7C573CCF4740   76003.0   66681.0  24051.0  19346.0  18494.0  11194.0   
22     7C573CCF47D0   81060.0   72905.0  26018.0  25562.0  24318.0  14979.0   
23     7C573CCF47E6   72128.0   65375.0  18584.0  16211.0  12094.0   6818.0   
24     7C573CCF484E   26842.0   24469.0   9976.0  12316.0  13458.0  10522.0   
25     7C573CCF48D4  132817.0  110696.0  43540.0  40069.0  35305.0  25186.0   
26     7C573CCF497E  163978.0  135579.0  44138.0  38868.0  35903.0  26022.0   
27     7C573CCF49AA   50293.0   48092.0  20407.0  15643.0  14798.0  10393.0   
28     7C573CCF49C0   32101.0   34664.0  17889.0  12176.0  10952.0   6521.0   
29     7C573CCF49F8   24864.0   23230.0   7085.0   7044.0   6206.0   4056.0   
30     7C573CCF4A94   22169.0   28803.0   8124.0   9614.0   7913.0   5156.0   
31     7C573CCF4AAE    7962.0    9626.0   3973.0   4312.0   3800.0   2262.0   
32     7C573CCF4AD6   17270.0   18092.0   5173.0   5838.0   4443.0   3144.0   
33     7C573CCF4AE6  107771.0  101106.0  53049.0  46726.0  42865.0  35011.0   
34     7C573CCF4AEE  115848.0  104899.0  38505.0  29387.0  24482.0  17442.0   
35     7C573CCF4B2E   17526.0   19015.0   7553.0   7629.0   6369.0   4509.0   
36     7C573CCF4B36  261671.0  226013.0  80928.0  79556.0  71015.0  53603.0   
37     7C573CCF4B7C   10769.0   13305.0   5001.0   5328.0   5117.0   3396.0   
38     7C573CCF4B80   48779.0   49645.0  19374.0  19151.0  17386.0  12302.0   
39     7C573CCF4B98   24253.0   24083.0  12200.0  12174.0  11322.0   7495.0   
40     7C573CCF4CCC  131014.0  113224.0  29618.0  24793.0  24188.0  17222.0   
41     7C573CCF4CE8   72957.0   65298.0  30497.0  27830.0  28180.0  21000.0   
42     7C573CCF4D0C   35353.0   36868.0  17973.0  15770.0  16659.0  13137.0   
43     7C573CCF4D6A   46592.0   42563.0  22899.0  15368.0  13295.0   9269.0   
44     7C573CCF4D74   50730.0   43356.0  12542.0  10677.0  10245.0   7790.0   
45     7C573CCF4E8C   49388.0   43854.0  18949.0  18593.0  15896.0  11802.0   
46     7C573CCF4EA6   44836.0   49516.0  18918.0  16074.0  14595.0   8993.0   
47     7C573CCF4ECC   52596.0   54984.0  21111.0  19933.0  19126.0  13892.0   
48     7C573CCF4EF2   57946.0   52378.0  22812.0  

In [116]:
pdf1 = pdf1[[('설치장소명', ''), ('설치위치', ''), ('MAC', ''),(     2021,  1),(     2021,  2),(     2021,  3),(     2021,  4),
(     2021,  5),(     2021,  6),(     2021,  7),(     2021,  8),(     2021,  9),(     2021, 10),(     2021, 11),(     2021, 12),
(     2022,  3),(     2022,  4),(     2022,  5),(     2022,  6),(     2022,  7),(     2022,  8),(     2022,  9),(     2022, 10)]]

In [119]:
pdf1.fillna(0, inplace=True)

In [125]:
p_l = [(     2021,  1),(     2021,  2),(     2021,  3),(     2021,  4),
(     2021,  5),(     2021,  6),(     2021,  7),(     2021,  8),(     2021,  9),(     2021, 10),(     2021, 11),(     2021, 12),
(     2022,  3),(     2022,  4),(     2022,  5),(     2022,  6),(     2022,  7),(     2022,  8),(     2022,  9),(     2022, 10)]

col_astype = dict()
for n in p_l:
    col_astype[n] = int

In [126]:
pdf1 = pdf1.astype(col_astype)

In [128]:
pdf1.to_excel('./전처리/와이파이_전체_이용자.xlsx')

In [50]:
df_2021_group = pd.DataFrame(concat_df.groupby(['yaer', 'MAC'])['user_count'].sum())
df_2021_group.reset_index(inplace=True)

In [150]:
wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])

In [53]:
df_2021_group['X'] = None
df_2021_group['Y'] = None

for n in tqdm(range(len(df_2021_group))):
    df_2021_group['X'][n] = wk.loc[df_2021_group['MAC'][n]]['경도']
    df_2021_group['Y'][n] = wk.loc[df_2021_group['MAC'][n]]['위도']

  0%|          | 0/53 [00:00<?, ?it/s]

In [55]:
df_2021_group.to_csv('./전처리/와이파이_이용자/2021.csv', encoding='cp949', index=False)

In [151]:
df_2022_group = pd.DataFrame(concat_df.groupby(['yaer', 'MAC'])['user_count'].sum())
df_2022_group.reset_index(inplace=True)

In [153]:
df_2022_group['X'] = None
df_2022_group['Y'] = None

for n in tqdm(range(len(df_2022_group))):
    df_2022_group['X'][n] = wk.loc[df_2022_group['MAC'][n]]['경도']
    df_2022_group['Y'][n] = wk.loc[df_2022_group['MAC'][n]]['위도']

  0%|          | 0/53 [00:00<?, ?it/s]

In [154]:
df_2022_group.to_csv('./전처리/와이파이_이용자/2022년/2022.csv', encoding='cp949', index=False)

In [63]:
df_2021.head()

Path_ID  CLIENTMAC  MAC_add      STARTTIMESTAMP        ENDTIMESTAMP  \
0        0  000000000       31 2021-01-01 12:20:00 2021-01-01 12:30:00   
1        1  000000000        7 2021-01-02 16:40:00 2021-01-02 16:50:00   
2        2  000000000       52 2021-01-02 18:30:00 2021-01-02 18:40:00   
3        3  000000000       42 2021-01-02 22:20:00 2021-01-02 22:30:00   
4        4  000000000       61 2021-01-03 14:10:00 2021-01-03 14:20:00   

   Previous_Path  Next_Path  Next_MAC  
0            NaN        NaN       NaN  
1            NaN        NaN       NaN  
2            NaN        NaN       NaN  
3            NaN        NaN       NaN  
4            NaN        NaN       NaN

In [64]:
df_2021.columns

Index(['Path_ID', 'CLIENTMAC', 'MAC_add', 'STARTTIMESTAMP', 'ENDTIMESTAMP',
       'Previous_Path', 'Next_Path', 'Next_MAC'],
      dtype='object')

In [67]:
def month_split_2(DF, year):
    
    if year == 2021:
        month_list = [n for n in range(1,13)]
    elif year == 2022:
        month_list = [n for n in range(3,11)]
    
    
    global concat_df
    for n in tqdm(month_list):
        
        ye = []
        mon = []
        con =[]
        mc=[]
        
        if n <= 9:
            num = '0' + str(n)
        else:
            num = str(n)

        # 기간설정(월별)
        if n != month_list[-1]:
            df = DF[(DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (DF['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{n+1}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S'))]
        else:
            df = DF[DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00', 
                                                              '%Y-%m-%d %H:%M:%S')]
            
        
        df = pd.DataFrame(df.groupby(['MAC_add'])['CLIENTMAC'].value_counts())
        df.reset_index(inplace=True)
        df_drop_list = list(df[df['CLIENTMAC']>2].index)
        df.drop(df_drop_list, axis=0, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        
        
        
        for mac in list(df['MAC_add'].unique()):
            
            
            df_d = df[df['MAC_add']==mac]
            
            ye.append(year)
            mon.append(n)
            mc.append(mac)
            con.append(len(df_d['CLIENTMAC'].unique()))
        
        
        sum_list = []
        for a,b,c,d in zip(ye, mon, mc, con):
            sum_list.append([a,b,c,d])    
        
        df_new = pd.DataFrame(sum_list, columns=['yaer', 'month', 'MAC', 'user_count'])
        
        if n == month_list[0]:
            concat_df = df_new
            continue
        concat_df = pd.concat([concat_df, df_new])
    return concat_df

In [ ]:
pd.DataFrame(df_2021.groupby(['MAC_add'])['CLIENTMAC'].value_counts())

In [73]:
df_2021.groupby(['MAC_add'])['CLIENTMAC'].value_counts()

MemoryError: Unable to allocate 75.1 MiB for an array with shape (9837806,) and data type int64

In [68]:
month_split_2(df_2021,2021)

  0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 